In [ ]:
import tensorflow as tf
save_path = "C:/data/result/saved_dataset"
def load_dataset(save_path):
    """
    从指定路径加载 TensorFlow 数据集。
    
    Args:
        save_path: 数据集保存路径。
    Returns:
        加载的 tf.data.Dataset 对象。
    """
    dataset = tf.data.experimental.load(save_path)
    print(f"数据集已从 {save_path} 加载")
    return dataset

# 加载数据集
dataset = load_dataset(save_path)

# 假设 dataset 是已经处理好的 tf.data.Dataset 对象
# 将 dataset 分为训练集和验证集
train_split = 0.8  # 80% 数据用于训练
val_split = 1 - train_split

# 获取总样本数
total_samples = len(list(dataset))
train_size = int(total_samples * train_split)

# 分割数据集
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

# 配置训练和验证集
train_ds = train_ds.shuffle(buffer_size=1000).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(8).prefetch(tf.data.AUTOTUNE)

del dataset
import gc
gc.collect()  # 强制进行垃圾回收

In [ ]:
import numpy as np
print("训练集标签分布:", np.unique(train_ds, return_counts=True))
print("验证集标签分布:", np.unique(val_ds, return_counts=True))


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import einops
# 启用 XLA
tf.config.optimizer.set_jit(True)

# 检查 GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"Using GPU: {physical_devices}")
else:
    print("No GPU found. Using CPU.")

# 定义自定义的硬连线层
class HardwiredLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(HardwiredLayer, self).__init__(**kwargs)

    def call(self, video):
        """
        从每个视频帧提取灰度、梯度等特征，拼接成一个张量。
        """
        # 假设 'video' 的形状为 (batch, time, height, width, channels)
        batch, time, height, width, channels = video.shape

        # 将视频的时间维度拆分开来，逐帧处理
        def process_frame(frame):
            # 灰度图：将每帧转化为灰度图（3通道转1通道）
            gray_frame = tf.image.rgb_to_grayscale(frame)

            # 计算x, y方向的梯度
            grad_x = tf.image.sobel_edges(frame)[:,:,:,:,0]  # sobel梯度计算，提取x方向的梯度
            grad_y = tf.image.sobel_edges(frame)[:,:,:,:,1]  # sobel梯度计算，提取y方向的梯度

            # 拼接特征（灰度 + x方向梯度 + y方向梯度）
            features = tf.concat([gray_frame, grad_x, grad_y], axis=-1)  # 在最后一个通道维度拼接
            return features

        # 逐帧处理所有时间步
        video_features = tf.map_fn(process_frame, video, dtype=tf.float32)

        return video_features


# 定义3D卷积层，考虑时间序列
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding, **kwargs):
        super().__init__(**kwargs)
        self.seq = keras.Sequential([  
            # 空间维度卷积
            layers.Conv3D(filters=filters,
                          kernel_size=(1, kernel_size[1], kernel_size[2]),  # 只对空间进行卷积
                          padding=padding),
            # 时间维度卷积
            layers.Conv3D(filters=filters, 
                          kernel_size=(kernel_size[0], 1, 1),  # 只对时间维度进行卷积
                          padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# 定义残差模块
class ResidualMain(keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.seq = keras.Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

# 定义视频尺寸调整层
class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width, **kwargs):
        super().__init__(**kwargs)
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# 定义模型
def build_model(input_shape):
    input = layers.Input(shape=input_shape)  # 输入层，形状为 (时间步数, 高度, 宽度, 通道数)
    x = input

    # 第一层：硬连线层，提取视频帧特征（灰度 + 梯度）
    x = HardwiredLayer()(x)

    # 第一层卷积：空间 + 时间的卷积
    x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = ResizeVideo(HEIGHT//2, WIDTH//2)(x)

    # 第一个残差块
    x = ResidualMain(filters=16, kernel_size=(3, 3, 3))(x)
    x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

   

    # 第二个残差块
    x = ResidualMain(filters=32, kernel_size=(3, 3, 3))(x)
    x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)
     # 第一个时间卷积层：进一步提取时间特征
    x = Conv2Plus1D(filters=32, kernel_size=(3, 5, 5), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 第三个残差块
    x = ResidualMain(filters=64, kernel_size=(3, 3, 3))(x)
    x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)



    # 第四个残差块
    x = ResidualMain(filters=128, kernel_size=(3, 3, 3))(x)

    # 全局平均池化和分类
    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(15)(x)  # 输出15类分类任务

    # 定义模型
    model = keras.Model(inputs=input, outputs=x)
    return model

# 假设高度和宽度为224
HEIGHT = 224
WIDTH = 224

# 定义输入的形状，假设20帧，每帧224x224的图像，RGB通道
input_shape = (20, HEIGHT, WIDTH, 3)

# 设置初始学习率
initial_learning_rate = 0.0002
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
# 使用Adam优化器并设置初始学习率
optimizer = Adam(learning_rate=initial_learning_rate)

# 定义学习率调整回调
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # 监控验证集的损失
    factor=0.5,          # 每次衰减的比例
    patience=3,          # 若验证集损失在连续3个epoch内不再改善，才衰减学习率
    verbose=1            # 打印学习率调整信息
)


# 构建模型
model = build_model(input_shape)

# 编译模型
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

# 打印模型摘要
model.summary()


Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use fn_output_signature instead
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 224, 224, 3)  0         
                             ]                                   
                                                                 
 hardwired_layer (HardwiredL  (None, 20, 224, 224, 7)  0         
 ayer)                                                           
                                                                 
 conv2_plus1d (Conv2Plus1D)  (None, 20, 224, 224, 16)  6288      
                                                                 
 batch_normalization (BatchN  (None, 20, 224, 224, 16)  64       
 ormalization)                                                   
                                                        

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print(tf.test.is_gpu_available())
# 列出所有物理 GPU 设备
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))



In [3]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',        # 监控的指标，通常选择验证集的损失值（val_loss）
    patience=10,                # 允许验证集损失在5个epoch内没有改善
    restore_best_weights=True, # 如果训练提前停止，恢复验证集损失最小的模型权重
    verbose=1                  # verbose控制输出的详细程度，1表示在停止时输出提示信息
)



In [ ]:
# 训练模型
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[ lr_scheduler,early_stop]
)